In [41]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [42]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:09<00:00, 1041.05it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



{'basic': {'expected_sample_burst_size': 3.004903369494935,
           'expected_sample_data_size': 16.48236223866657,
           'expected_time_to_recover': 99.2007,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.1612891844513194,
             'bit_redundancy': 416.0035,
             'ideal_bit_efficency': 0.19069385392787552,
             'ideal_bit_redundancy': 339.52059991538954,
             'ideal_packet_efficiency': 0.9534692696393776,
             'ideal_packet_redundancy': 67.9041199830779,
             'ideal_time_to_recover': 83.9041199830779,
             'ideal_transmitted_bits': 419.52059991538954,
             'packet_efficiency': 0.806445922256597,
             'packet_redundancy': 83.2007,
             'permeability': 0.8458016927610179,
             'saturation': 16.0,
             'transmitted_bits': 496.0035}}
Found implementation: anfext2
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1844.23it/s]


{'basic': {'expected_sample_burst_size': 2.987622175933344,
           'expected_sample_data_size': 16.519548149335044,
           'expected_time_to_recover': 39.425999999999995,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.4058235682037235,
             'bit_redundancy': 117.12999999999997,
             'ideal_bit_efficency': 0.4792182809961945,
             'ideal_bit_redundancy': 86.93853964355606,
             'ideal_packet_efficiency': 2.3960914049809725,
             'ideal_packet_redundancy': 17.387707928711208,
             'ideal_time_to_recover': 33.38770792871121,
             'ideal_transmitted_bits': 166.93853964355606,
             'packet_efficiency': 2.0291178410186173,
             'packet_redundancy': 23.425999999999995,
             'permeability': 0.8468449228608333,
             'saturation': 16.0,
             'transmitted_bits': 197.12999999999997}}
Found implementation: anfext2l
  First time seen. Benchmarking.


...

{'basic': {'expected_sample_burst_size': 3.0176093188333257,
           'expected_sample_data_size': 16.531373708877464,
           'expected_time_to_recover': 32.7137,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15284116440512688,
             'bit_redundancy': 138.56850000000003,
             'ideal_bit_efficency': 0.18074053502806378,
             'ideal_bit_redundancy': 113.31982956185354,
             'ideal_packet_efficiency': 0.9037026751403189,
             'ideal_packet_redundancy': 22.663965912370706,
             'ideal_time_to_recover': 27.663965912370706,
             'ideal_transmitted_bits': 138.31982956185354,
             'packet_efficiency': 0.7642058220256345,
             'packet_redundancy': 27.713700000000003,
             'permeability': 0.8456385524220954,
             'saturation': 5.0,
             'transmitted_bits': 163.56850000000003}}
Found implementation: single_binary_fullrank
  First time seen. Benchmar

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 3253.67it/s]


{'basic': {'expected_sample_burst_size': 3.012119771863118,
           'expected_sample_data_size': 16.378247782002536,
           'expected_time_to_recover': 13.257200000000001,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.35080415666617615,
             'bit_redundancy': 43.03259567122585,
             'ideal_bit_efficency': 0.41532046820391505,
             'ideal_bit_redundancy': 32.73565980123434,
             'ideal_packet_efficiency': 2.076602341019575,
             'ideal_packet_redundancy': 6.547131960246869,
             'ideal_time_to_recover': 11.1978128260017,
             'ideal_transmitted_bits': 55.989064130008494,
             'packet_efficiency': 1.7540207833308807,
             'packet_redundancy': 8.60651913424517,
             'permeability': 0.8446589646382116,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.286}}
Found implementation: single_binary_fullrank2
  First 

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 2919.13it/s]


{'basic': {'expected_sample_burst_size': 2.960796762161733,
           'expected_sample_data_size': 16.524402825172604,
           'expected_time_to_recover': 13.776200000000001,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3375880769555342,
             'bit_redundancy': 45.62759567122585,
             'ideal_bit_efficency': 0.3980761742120177,
             'ideal_bit_redundancy': 35.16105459935364,
             'ideal_packet_efficiency': 1.990380871060088,
             'ideal_packet_redundancy': 7.03221091987073,
             'ideal_time_to_recover': 11.68289178562556,
             'ideal_transmitted_bits': 58.41445892812779,
             'packet_efficiency': 1.687940384777671,
             'packet_redundancy': 9.12551913424517,
             'permeability': 0.8480489384319013,
             'saturation': 4.650680865754831,
             'transmitted_bits': 68.881}}
